https://stepik.org/lesson/1370101/step/4

In [ ]:
import numpy as np

np.random.seed(0)

# исходные параметры распределений двух классов
mean1 = [1, -2]
mean2 = [1, 3]
r = 0.7
D = 2.0
V = [[D, D * r],
     [D * r, D]]

# параметры Py1, L1
Py1, L1 = 0.5, 1      # вероятности появления классов
Py2, L2 = 1 - Py1, 1  # и величины штрафов неверной классификации

# моделирование обучающей выборки
N = 1000
x1 = np.random.multivariate_normal(mean1, V, N).T
x2 = np.random.multivariate_normal(mean2, V, N).T

x_train = np.hstack([x1, x2]).T
y_train = np.hstack([np.ones(N) * -1, np.ones(N)])

# вычисление оценок МО и ковариационной матрицы
mm1 = np.mean(x1.T, axis=0)
mm2 = np.mean(x2.T, axis=0)

a = np.hstack([(x1.T - mm1).T, (x2.T - mm2).T])
VV = np.array([[np.dot(a[0], a[0]) / (2*N), np.dot(a[0], a[1]) / (2*N)],
                [np.dot(a[1], a[0]) / (2*N), np.dot(a[1], a[1]) / (2*N)]])

# Функции:
# вычисление вероятности
b = lambda x, v, m, l, py: np.log(l * py) - 0.5 * m @ np.linalg.inv(v) @ m + x @ np.linalg.inv(v) @ m
# argmax от вероятностей
am = lambda x: np.argmax([b(x, VV, mm1, L1, Py1), b(x, VV, mm2, L2, Py2)]) * 2 - 1 # классификатор

# Предсказание по модели
predict = [am(x) for x in x_train]
predict[0], predict[-1]

# Качество
Q = sum(predict != y_train)
Q

8

In [ ]:
# Виталий Золотов https://stepik.org/lesson/1370101/step/4?discussion=10354893&thread=solutions&unit=1386291

import numpy as np

np.random.seed(0)

# Исходные параметры распределений двух классов
mean1 = [1, -2]
mean2 = [1, 3]
r = 0.7
D = 2.0
V = [[D, D * r], [D * r, D]]

# Моделирование обучающей выборки
N = 1000
x1 = np.random.multivariate_normal(mean1, V, N).T
x2 = np.random.multivariate_normal(mean2, V, N).T

# Объединение выборок
x_train = np.hstack([x1, x2]).T
y_train = np.hstack([np.ones(N) * -1, np.ones(N)])

# Вычисление оценок математического ожидания и ковариационной матрицы
mu1 = np.mean(x1.T, axis=0)  # Математическое ожидание первого класса
mu2 = np.mean(x2.T, axis=0)  # Математическое ожидание второго класса

# Объединяем данные для вычисления ковариационной матрицы
data = np.vstack([x1.T - mu1, x2.T - mu2])

# Вычисляем ковариационную матрицу
VV = np.cov(data.T)

# Обратная ковариационная матрица
Sigma_inv = np.linalg.inv(VV)

# Параметры для дискриминанта Фишера
P_y1 = 0.5  # априорная вероятность класса -1
P_y2 = 1 - P_y1  # априорная вероятность класса -1
lambda_y1, lambda_y2 = 1, 1  # величины штрафов неверной классификации

# Вычисление d1 и d2 для всех обучающих образцов
alpha1 = Sigma_inv @ mu1
alpha2 = Sigma_inv @ mu2
beta1 = np.log(lambda_y1 * P_y1) - 0.5 * mu1 @ Sigma_inv @ mu1
beta2 = np.log(lambda_y2 * P_y2) - 0.5 * mu2 @ Sigma_inv @ mu2
d1 = x_train @ alpha1 + beta1 # [длинный список] - 4.68
d2 = x_train @ alpha2 + beta2

# Предсказания классов
predict = np.where(d1 > d2, -1, 1)

# Подсчет количества ошибок
Q = np.sum(predict != y_train)